In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os
from ipywidgets import interact
import numpy as np
import albumentations as A

In [14]:
csv_path = '/opt/ml/input/code/submission'
csv_fname = 'upernet_swin_val(pretrained).csv'
csv_fpath = os.path.join(csv_path, csv_fname)

base_path = '/opt/ml/mmseg_dataset'
phase = 'val'
img_bpath = os.path.join(base_path, 'images', phase)
ann_bpath = os.path.join(base_path, 'annotations', phase)

In [6]:
prediction = pd.read_csv(csv_fpath)

,image_id,PredictionString
0,batch_01_vt/0002.jpg,0 0 0 0 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 ...
1,batch_01_vt/0009.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2,batch_01_vt/0018.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
3,batch_01_vt/0019.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4,batch_01_vt/0030.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...


In [13]:
test = '1 2 3 4 5 6'
np.array(test.split(' ')).reshape(-1, 3)

array([['1', '2', '3'],
       ['4', '5', '6']], dtype='<U1')

In [65]:
size = 256
transform = A.Compose([A.Resize(size, size)])

PALETTE = [[0, 0, 0], [192, 0, 128], [0, 128, 192], [0, 128, 64],
               [128, 0, 0], [64, 0, 128], [64, 0, 192], [192, 128, 64],
               [192, 192, 128], [64, 64, 128], [128, 0, 192]]
CLASSES = ('Background', 'General trash', 'Paper', 'Paper pack', 'Metal', 'Glass', 
            'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing')

In [52]:
test = np.array([[1,2,3], [4,5,6], [7, 8, 9]])
test.shape
test[3, test.shape]
# mask[0,test==1] = PALETTE[0][0]
# mask[1, test==1] = PALETTE[0][1]
# mask[2,test==1] = PALETTE[0][2]
# mask

IndexError: index 3 is out of bounds for axis 0 with size 3

In [107]:
@interact(idx=(0, prediction.shape[0]-1))
def ShowImg(idx):
    fname = prediction.iloc[idx]['image_id'].replace('/', '-')

    img = np.array(Image.open(os.path.join(img_bpath, fname)))
    gt = np.array(Image.open(os.path.join(ann_bpath, fname.replace('jpg', 'png'))))
    transforms = transform(image=img, mask=gt)

    resized_img = transforms['image']
    resized_gt = transforms['mask'].astype(int)
    predict = np.array(prediction.iloc[idx]['PredictionString'].split(' ')).reshape(256, 256).astype(int)
    
    predict_mask = np.zeros((3, 256, 256))
    gt_mask = np.zeros((3, 256, 256))
    color_label = np.zeros((3, 250, 250))
    

    fig, axes = plt.subplots(1, 4, figsize=(20, 10), sharey=True)
    for i in range(10):
        for j in range(3):
            predict_mask[j, predict==i] = PALETTE[i][j]
            gt_mask[j, resized_gt==i] = PALETTE[i][j]
            color_label[j, i*25:i*25+25] = PALETTE[i][j]
        plt.sca(axes[3])
        plt.text(0,i*25+20, color='white', s=CLASSES[i], fontsize=15)
    
    axes[0].imshow(resized_img)
    axes[1].imshow((gt_mask.transpose(1, 2, 0))/255.0)
    axes[2].imshow(predict_mask.transpose(1, 2, 0)/255.0)
    axes[3].imshow(color_label.transpose(1, 2, 0)/255.0)
    axes[0].set_title('img')
    axes[1].set_title('gt')
    axes[2].set_title('predict')
    axes[3].set_title('color label')


interactive(children=(IntSlider(value=327, description='idx', max=654), Output()), _dom_classes=('widget-inter…